In [ ]:
import os
import shutil
import random
PROJ_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(PROJ_DIR, "src"))

from pkg.utils.utils import *
from pkg.planning.filtering.lattice_model.data_utils import *

DATA_PATH = os.path.join(PROJ_DIR, "data")
LAT_DATA_PATH = os.path.join(DATA_PATH, "latticized")
MODEL_PATH = os.path.join(PROJ_DIR, "model")
LAT_MODEL_PATH = os.path.join(MODEL_PATH,"latticized")
try_mkdir(MODEL_PATH)
try_mkdir(LAT_MODEL_PATH)
GRASP_FOLDER = "grasp"
ARM10_FOLDER = "arm_10"
ARM05_FOLDER = "arm_05"
FULLS_FOLDER = "full_scene"
GSCENE_FOLDER = "gscene"

VISUALIZE = False

In [ ]:
ROBOT_TYPE_NAME="indy7gripper"
ROBOT_DATA_ROOT = os.path.join(LAT_DATA_PATH, ROBOT_TYPE_NAME)
ROBOT_DATA_ROOT_FAILMORE = ROBOT_DATA_ROOT+"-failmore"
ROBOT_MODEL_ROOT =  os.path.join(LAT_MODEL_PATH, ROBOT_TYPE_NAME)

In [ ]:
dataset_list = sorted(os.listdir(ROBOT_DATA_ROOT))
DATASET_TRAIN_FILTERED = dataset_list[:10]
DATASET_TEST_FILTERED = dataset_list[10:15]
print(DATASET_TRAIN_FILTERED)
print(DATASET_TEST_FILTERED)

dataset_list_fails = sorted([folder for folder 
                             in os.listdir(ROBOT_DATA_ROOT_FAILMORE) 
                             if not folder.startswith(".")])
DATASET_TRAIN_FAILMORE = dataset_list_fails[:10]
DATASET_TEST_FAILMORE = dataset_list_fails[10:15]
print(DATASET_TRAIN_FAILMORE)
print(DATASET_TEST_FAILMORE)

In [ ]:
DATA_ROOT = ROBOT_DATA_ROOT
DATASET = DATASET_TRAIN_FILTERED

def get_data_pairs(data_root, datasets):
    data_pairs = []
    for dataset in datasets:
        file_list = sorted(os.listdir(os.path.join(data_root, dataset, GRASP_FOLDER)))
        for file in file_list:
            data_pairs.append(
                (os.path.join(data_root, dataset, GRASP_FOLDER, file), 
                 os.path.join(data_root, dataset, ARM10_FOLDER, file), 
                 os.path.join(data_root, dataset, ARM05_FOLDER, file), 
                 os.path.join(data_root, dataset, FULLS_FOLDER, file), 
                 os.path.join(data_root, dataset, GSCENE_FOLDER, file)))
    return data_pairs

In [ ]:
data_pairs_train_filtered = get_data_pairs(ROBOT_DATA_ROOT, DATASET_TRAIN_FILTERED)
print("train set: {}".format(len(data_pairs_train_filtered)))

data_pairs_test_filtered = get_data_pairs(ROBOT_DATA_ROOT, DATASET_TEST_FILTERED)
print("test set: {}".format(len(data_pairs_test_filtered)))

data_pairs_train_failmore = get_data_pairs(ROBOT_DATA_ROOT_FAILMORE, DATASET_TRAIN_FAILMORE)
print("train fail set: {}".format(len(data_pairs_train_failmore)))

data_pairs_test_failmore = get_data_pairs(ROBOT_DATA_ROOT_FAILMORE, DATASET_TEST_FAILMORE)
print("test fail set: {}".format(len(data_pairs_test_failmore)))

In [ ]:
from pkg.planning.filtering.lattice_model.latticizer_py import *
from pkg.planning.constraint.constraint_common import *
import itertools

ltc_full = Latticizer_py(WDH=(3, 3, 3), L_CELL=0.05, OFFSET_ZERO=(1.5, 1.5, 1.5))
ltc_effector = Latticizer_py(WDH=(1, 1, 1), L_CELL=0.05, OFFSET_ZERO=(0.5, 0.5, 0.5))
ltc_arm_05 = Latticizer_py(WDH=(2, 2, 2), L_CELL=0.05, OFFSET_ZERO=(0.5, 1.0, 1.0))
ltc_arm_10 = Latticizer_py(WDH=(2, 2, 2), L_CELL=0.10, OFFSET_ZERO=(0.5, 1.0, 1.0))


In [ ]:
from pkg.utils.test_scripts import *

ROBOT_TYPE = {e.name: e for e in RobotType}[ROBOT_TYPE_NAME]
ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, HOME_POSE, GRIP_DEPTH = get_single_robot_params(ROBOT_TYPE)
s_builder, pscene = prepare_single_robot_scene(ROBOT_TYPE, ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, VISUALIZE=VISUALIZE)
crob, gscene = pscene.combined_robot, pscene.gscene
crob.home_pose = HOME_POSE
crob.home_dict = list2dict(HOME_POSE, gscene.joint_names)
HOME_DICT = crob.home_dict
shoulder_link = gscene.urdf_content.joint_map[gscene.joint_names[1]].child
shoulder_height = get_tf(shoulder_link, HOME_DICT, gscene.urdf_content)[2,3]
gtems_robot = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)


from pkg.planning.filtering.grasp_filter import GraspChecker
gcheck = GraspChecker(pscene)

In [ ]:
for data_pairs in [data_pairs_train_filtered, data_pairs_test_filtered, data_pairs_train_failmore, data_pairs_test_failmore]:
    for data_pair in data_pairs:
        grasp_file, arm10_file, arm5_file, fscene_file, gscene_file =data_pair
        print("======== {} ========".format(gscene_file))
        grasp_dict_load = load_pickle(grasp_file)
        arm10_dict_load = load_pickle(arm10_file)
        arm05_dict_load = load_pickle(arm5_file)
        fscene_dict_load = load_pickle(fscene_file)
        scene_dat = load_pickle(gscene_file)
        T_loal = scene_dat["T_loal"]
        pscene.clear_subjects()
        gscene.clear_non_fixed()
        gscene.clear_link("base_link")
        initial_state = load_saved_scene(pscene, gscene_file, VISUALIZE=VISUALIZE, 
                                         GRIP_DEPTH=GRIP_DEPTH, CLEARANCE = 1e-3)

        success_reach, success_retrieve = grasp_dict_load["reach"], grasp_dict_load["retrieve"]
        time_reach, success_retrieve = grasp_dict_load["reach_time"], grasp_dict_load["retrieve_time"]

        grasp_dict = {}
        arm_05_dict = {}
        arm_10_dict = {}
        full_scene_dict = {}
        ltc_effector.clear()
        ltc_arm_05.clear()
        ltc_arm_10.clear()
        ltc_full.clear()

        ##################################################
        ################ CHECK REGENERATION ################

        actor = pscene.actor_dict['grip0']
        actor.redundancy['w'] = (-np.deg2rad(45), np.deg2rad(45))# default redundancy changed
        T_ba = np.matmul(T_loal, actor.Toff_lh)
        obj, handle = find_match(pscene, actor, T_ba, crob.home_dict)

        actor_vertinfo_list, object_vertinfo_list, actor_Tinv_dict, object_Tinv_dict = \
                                            gcheck.get_grasping_vert_infos(actor, obj, T_loal, HOME_DICT)

        if all([ROBOT_NAME in lname for lname in actor_Tinv_dict.keys()]):
            tool_Tinv_dict = actor_Tinv_dict
            tool_vertinfo_list = actor_vertinfo_list
            target_vertinfo_list = object_vertinfo_list
            T_end_effector = T_loal
        elif all([ROBOT_NAME in lname for lname in object_Tinv_dict.keys()]):
            tool_Tinv_dict = object_Tinv_dict
            tool_vertinfo_list = object_vertinfo_list
            target_vertinfo_list = actor_vertinfo_list
            T_end_effector = SE3_inv(T_loal)
        else:
            raise("Invaild TOOL_LINK_BUNDLE")
        T_end_joint = T_end_effector

        r, th, h = cart2cyl(*T_end_effector[:3,3])
        Tref = SE3(Rot_axis(3, th), T_end_effector[:3,3])
        Tref_base = SE3(Tref[:3,:3], (0, 0, shoulder_height))
        
#         obj_names = [obj.geometry.name] + obj.geometry.children
#         target_names= [item[0] for item in target_vertinfo_list if item[0] not in obj_names]
#         tool_names = [item[0] for item in tool_vertinfo_list]

#         ltc_effector.convert_vertices(tool_vertinfo_list, Tref=Tref)
#         ltc_effector.convert_vertices(target_vertinfo_list, Tref=Tref)
#         ltc_arm_05.convert([gtem for gtem in gscene if gtem.collision and gtem not in gtems_robot], HOME_DICT, Tref=Tref_base)
#         ltc_arm_10.convert([gtem for gtem in gscene if gtem.collision and gtem not in gtems_robot], HOME_DICT, Tref=Tref_base)
#         ltc_full.convert([gtem for gtem in gscene if gtem.collision and gtem not in gtems_robot], HOME_DICT, Tref=Tref_base)
#         ltc_full.convert_vertices(actor_vertinfo_list, Tref=Tref_base)

#         grasp_dict["tar"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in target_names if tname in ltc_effector.coll_idx_dict])))
#         grasp_dict["tool"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in tool_names if tname in ltc_effector.coll_idx_dict])))
#         grasp_dict["obj"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in obj_names if tname in ltc_effector.coll_idx_dict])))
#         grasp_dict["T_end_effector"], grasp_dict["T_end_joint"], grasp_dict["Tref_base"]  = T_end_effector, T_end_joint, Tref_base
#         grasp_dict["reach"], grasp_dict["retrieve"] = success_reach, success_retrieve
#         grasp_dict["reach_time"], grasp_dict["retrieve_time"] = time_reach, success_retrieve

#         arm_05_dict["tar"] = sorted(set(itertools.chain(*[ltc_arm_05.coll_idx_dict[tname] for tname in target_names if tname in ltc_arm_05.coll_idx_dict])))
#         arm_05_dict["tool"] = sorted(set(itertools.chain(*[ltc_arm_05.coll_idx_dict[tname] for tname in tool_names if tname in ltc_arm_05.coll_idx_dict])))
#         arm_05_dict["T_end_effector"], arm_05_dict["T_end_joint"], arm_05_dict["Tref_base"] = T_end_effector, T_end_joint, Tref_base
#         arm_05_dict["reach"], arm_05_dict["retrieve"] = success_reach, success_retrieve

#         arm_10_dict["tar"] = sorted(set(itertools.chain(*[ltc_arm_10.coll_idx_dict[tname] for tname in target_names if tname in ltc_arm_10.coll_idx_dict])))
#         arm_10_dict["tool"] = sorted(set(itertools.chain(*[ltc_arm_10.coll_idx_dict[tname] for tname in tool_names if tname in ltc_arm_10.coll_idx_dict])))
#         arm_10_dict["T_end_effector"], arm_10_dict["T_end_joint"], arm_10_dict["Tref_base"] = T_end_effector, T_end_joint, Tref_base
#         arm_10_dict["reach"], arm_10_dict["retrieve"] = success_reach, success_retrieve

#         full_scene_dict["tar"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in target_names if tname in ltc_full.coll_idx_dict])))
#         full_scene_dict["tool"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in tool_names if tname in ltc_full.coll_idx_dict])))
#         full_scene_dict["obj"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in obj_names if tname in ltc_full.coll_idx_dict])))
#         full_scene_dict["T_end_effector"], full_scene_dict["T_end_joint"], full_scene_dict["Tref_base"] = T_end_effector, T_end_joint, Tref_base
#         full_scene_dict["reach"], full_scene_dict["retrieve"] = success_reach, success_retrieve

#         assert compare_dict(grasp_dict_load,grasp_dict)
#         assert compare_dict(arm05_dict_load,arm_05_dict)
#         assert compare_dict(arm10_dict_load,arm_10_dict)
#         assert compare_dict(fscene_dict_load,full_scene_dict)

        ################ CHECK REGENERATION ################
        ##################################################

        ##################################################
        ################## MAKE NEW DATA ##################

        (object_geo_list, object_T2end_dict), (actor_geo_list, actor_T2end_dict) = \
                        gcheck.get_geolist_tflist_pairs(actor, obj, HOME_DICT)

        (object_geo_list, object_T2end_dict), (actor_geo_list, actor_T2end_dict) = \
                        gcheck.get_geolist_tflist_pairs(actor, obj, HOME_DICT)


        obj_names = obj.geometry.get_family()

        group_name = ROBOT_NAME
        T_br = gscene.get_tf(crob.get_robot_base_dict()[group_name], Q=crob.home_dict,
                             from_link=obj.geometry.link_name)
        T_re = np.matmul(SE3_inv(T_br), T_loal)
        T_tool_to_rob = T_re
        tool_geo_list, tool_T2end_dict = actor_geo_list, actor_T2end_dict
        T_tar_to_rob = SE3_inv(T_br)
        target_geo_list, target_T2end_dict = object_geo_list, object_T2end_dict

        grasp_dict = {}
        arm_05_dict = {}
        arm_10_dict = {}
        full_scene_dict = {}
        ltc_effector.clear()
        ltc_arm_05.clear()
        ltc_arm_10.clear()
        ltc_full.clear()

        r, th, h = cart2cyl(*T_tool_to_rob[:3, 3])
        T_rl = SE3(Rot_axis(3, th), T_re[:3, 3])  # in robot base link coordinate
        target_names = [gtem.name for gtem in target_geo_list if gtem.name not in obj_names]
        tool_names = [gtem.name for gtem in tool_geo_list]

        ## Convert effector
        T_gl_list = []
        gtem_list = target_geo_list + tool_geo_list
        for gtem in gtem_list:
            if gtem.link_name in tool_T2end_dict:
                T_rg = matmul_series(T_tool_to_rob, tool_T2end_dict[gtem.link_name], gtem.Toff)
            else:
                T_rg = matmul_series(T_tar_to_rob, target_T2end_dict[gtem.link_name], gtem.Toff)
            T_lg = np.matmul(SE3_inv(T_rl), T_rg)
            T_gl = SE3_inv(T_lg)
            T_gl_list.append(T_gl)

        ltc_effector.convert_vertices_approx(gtem_list, T_gl_list)

        ## Convert env
        link_env = [lname for lname in gscene.link_names
                    if lname not in pscene.robot_chain_dict[group_name]["link_names"]]
        gtems_env = [gtem for gtem in gscene
                     if gtem.collision and gtem.link_name in link_env]
        Trl_base = SE3(T_rl[:3, :3], (0, 0, shoulder_height))  # in robot base link coordinate
        T_bl = np.matmul(T_br, Trl_base)
        T_lb = SE3_inv(T_bl)
        Tlist_env = {lname: gscene.get_tf(lname, crob.home_dict) for lname in link_env}
        T_gl_list_env = []
        for gtem in gtems_env:
            T_lg = matmul_series(T_lb, Tlist_env[gtem.link_name], gtem.Toff)
            T_gl = SE3_inv(T_lg)
            T_gl_list_env.append(T_gl)
        ltc_arm_05.convert_vertices_approx(gtems_env, T_gl_list_env)
        ltc_arm_10.convert_vertices_approx(gtems_env, T_gl_list_env)

        T_gl_list_tool = []
        for gtem in tool_geo_list:
            T_rg = matmul_series(T_tool_to_rob, tool_T2end_dict[gtem.link_name], gtem.Toff)
            T_lg = matmul_series(T_lb, T_br, T_rg)
            T_gl = SE3_inv(T_lg)
            T_gl_list_tool.append(T_gl)
        ltc_full.convert_vertices_approx(gtems_env, T_gl_list_env)
        ltc_full.convert_vertices_approx(tool_geo_list, T_gl_list_tool)

        grasp_dict["tar"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in target_names if tname in ltc_effector.coll_idx_dict])))
        grasp_dict["tool"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in tool_names if tname in ltc_effector.coll_idx_dict])))
        grasp_dict["obj"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in obj_names if tname in ltc_effector.coll_idx_dict])))
        grasp_dict["T_end_effector"], grasp_dict["T_end_joint"], grasp_dict["Tref_base"]  = T_loal, T_end_joint, Tref_base
        grasp_dict["reach"], grasp_dict["retrieve"] = success_reach, success_retrieve
        grasp_dict["reach_time"], grasp_dict["retrieve_time"] = time_reach, success_retrieve

        arm_05_dict["tar"] = sorted(set(itertools.chain(*[ltc_arm_05.coll_idx_dict[tname] for tname in target_names if tname in ltc_arm_05.coll_idx_dict])))
        arm_05_dict["tool"] = sorted(set(itertools.chain(*[ltc_arm_05.coll_idx_dict[tname] for tname in tool_names if tname in ltc_arm_05.coll_idx_dict])))
        arm_05_dict["T_end_effector"], arm_05_dict["T_end_joint"], arm_05_dict["Tref_base"] = T_loal, T_end_joint, Tref_base
        arm_05_dict["reach"], arm_05_dict["retrieve"] = success_reach, success_retrieve

        arm_10_dict["tar"] = sorted(set(itertools.chain(*[ltc_arm_10.coll_idx_dict[tname] for tname in target_names if tname in ltc_arm_10.coll_idx_dict])))
        arm_10_dict["tool"] = sorted(set(itertools.chain(*[ltc_arm_10.coll_idx_dict[tname] for tname in tool_names if tname in ltc_arm_10.coll_idx_dict])))
        arm_10_dict["T_end_effector"], arm_10_dict["T_end_joint"], arm_10_dict["Tref_base"] = T_loal, T_end_joint, Tref_base
        arm_10_dict["reach"], arm_10_dict["retrieve"] = success_reach, success_retrieve

        full_scene_dict["tar"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in target_names if tname in ltc_full.coll_idx_dict])))
        full_scene_dict["tool"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in tool_names if tname in ltc_full.coll_idx_dict])))
        full_scene_dict["obj"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in obj_names if tname in ltc_full.coll_idx_dict])))
        full_scene_dict["T_end_effector"], full_scene_dict["T_end_joint"], full_scene_dict["Tref_base"] = T_loal, T_end_joint, Tref_base
        full_scene_dict["reach"], full_scene_dict["retrieve"] = success_reach, success_retrieve

        save_pickle(grasp_file, grasp_dict)
        save_pickle(arm5_file, arm_05_dict)
        save_pickle(arm10_file, arm_10_dict)
        save_pickle(fscene_file, full_scene_dict)
        ################## MAKE NEW DATA ##################
        ##################################################

### debugging

In [ ]:
find_match(pscene, actor, T_ba, crob.home_dict)

In [ ]:
gscene.clear_highlight()

In [ ]:
gscene.add_highlight_axis("actor", "point", link_name="base_link", center=T_ba[:3,3], orientation_mat=T_ba[:3,:3])

In [ ]:
obj_0 = pscene.subject_dict['obj_0']
handle = obj_0.action_points_dict['obj_0_hdl_tp_a']
pscene.add_handle_axis("handle", handle)

In [ ]:
actor.check_type(handle)
binder_redundancy = actor.get_redundancy()
binder_T = T_ba
handle_T = handle.get_tf_handle(crob.home_dict)
handle_redundancy = handle.get_redundancy()
margin_mat = get_binding_margins(handle_T, binder_T, handle_redundancy, binder_redundancy)

In [ ]:
np.min(margin_mat)

In [ ]:
### Comparing errors

In [ ]:
def compare_idc(name, idc_strict, idc_approx, raise_error=False, allow_ratio=0.1):
    diff_inv = list(set(idc_approx) - set(idc_strict))
    if len(diff_inv) > 0: 
        msg = name + " cell over {} / {}".format(len(diff_inv), len(idc_strict))
        if raise_error:
            raise(RuntimeError(msg))
        else:
            print(msg)
            return list(diff_inv)
    if len(idc_strict)==0:
        return
    diff = tuple(np.transpose(sorted(set(idc_strict) - set(idc_approx))))

    if float(len(diff))/len(idc_strict) > allow_ratio: 
        msg =name + " diff too much {}".format(round(
            float(len(diff))/len(idc_strict), 3))
        if raise_error:
            raise(RuntimeError(msg))
        else:
            print(msg)      
            return list(diff  )

In [ ]:
compare_idc("grasp_tar", grasp_dict_load['tar'], grasp_dict['tar'])
compare_idc("grasp_tool", grasp_dict_load['tool'], grasp_dict['tool'])
compare_idc("grasp_obj", grasp_dict_load['obj'], grasp_dict['obj'])

diff_idc = compare_idc("arm05_tar", arm05_dict_load['tar'], arm_05_dict['tar'], allow_ratio=0.0)
compare_idc("arm05_tool", arm05_dict_load['tool'], arm_05_dict['tool'])

diff_idc = compare_idc("arm10_tar", arm10_dict_load['tar'], arm_10_dict['tar'], allow_ratio=0.0)
compare_idc("arm05_tool", arm10_dict_load['tool'], arm_10_dict['tool'])

diff_idc = compare_idc("fscene_tar", fscene_dict_load['tar'], full_scene_dict['tar'], allow_ratio=0.0)
diff_idc = compare_idc("fscene_tool", fscene_dict_load['tool'], full_scene_dict['tool'])
compare_idc("fscene_obj", fscene_dict_load['obj'], full_scene_dict['obj'])

In [ ]:
T_loal

In [ ]:
scene_dat = load_pickle(gscene_file)
T_loal = scene_dat["T_loal"]
        
diff_idc = grasp_dict_load['tar']
r, th, h = cart2cyl(*T_loal[:3, 3])
T_rl = SE3(Rot_axis(3, th), T_re[:3, 3])  # in robot base link coordinate

In [ ]:
ltc_centers = ltc_effector.get_centers().reshape((-1,3))
diff_centers = np.matmul(Trl_base[:3,:3], ltc_centers[diff_idc].transpose()).transpose() + Trl_base[:3,3]

In [ ]:
gscene.clear_highlight()

In [ ]:
for i_c, center in enumerate(diff_centers):
    gscene.add_highlight_axis("diff", "{:05}".format(i_c), link_name=crob.get_robot_base_dict()[ROBOT_NAME], 
                              center=center, orientation_mat=np.identity(3), axis=None)